In [ ]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from astropy import table
import cmasher as cmr
import dynamite as dyn
from plotbin import display_pixels

In [ ]:
# First, create a model's orblib
fname = 'user_test_config_ml.yaml'
c = dyn.config_reader.Configuration(fname,                        #   |
                                    reset_logging=True,           #   |
                                    user_logfile='test_nnls',     #   v
                                    reset_existing_output=True)   # 'False' saves time, use only if orblibs have been created with the current orblib_new_mirror
stars = c.system.get_unique_triaxial_visible_component()
parset = c.parspace.get_parset()
parset['ml'] = 1
model = dyn.model.Model(config=c, parset=parset)
model.setup_directories()
orblib = model.get_orblib()
print(f'{orblib.parset=}')
orblib.read_losvd_histograms()
print(f'{len(orblib.losvd_histograms)=}\t{len(orblib.intrinsic_masses)=}\t{len(orblib.projected_masses)=}\t{orblib.n_orbs=}')

In [ ]:
for h in orblib.losvd_histograms:
    print(f'{h}: {h.y.shape=}\n')
    mean = h.get_mean()
    sigma = h.get_sigma()
    if type(h.xedg)==tuple:
        print(f'{[x.shape for x in h.xedg]=}')
        print(f'{[x.shape for x in mean]=}\n{[x.shape for x in sigma]=}')
    else:
        print(f'{h.xedg.shape=}')
        print(f'{mean.shape=}\n{sigma.shape=}')
#    print(f'{h.get_normalisation().shape=}')
#    print(f'{np.sum(h.y)=}')
#    h.normalise()
#    print(f'{np.sum(h.y)=}')

In [ ]:
# plot the losvd and pm histograms' mean values
# mostly from plotter.py
orb_skip = 50
for i, orb_idx in enumerate(range(0, orblib.n_orbs, orb_skip)):

    def create_plot(dp_args, data):
        # get aperture and bin data
        x = dp_args['x']
        y = dp_args['y']
        dx = dp_args['dx']
        grid = dp_args['idx_bin_to_pix']
        angle_deg = dp_args['angle']
        # Only select the pixels that have a bin associated with them.
        s = np.ravel(np.where((grid >= 0)))
    
        #fhist, _ = np.histogram(grid[s], bins=len(data))
        #data = data / fhist
        # plot settings
        #data_min = min(data[grid[s]] / max(data))
        #data_max = max(data[grid[s]] / max(data))
        # The galaxy has NOT already rotated with PA to align major axis with x
        map1 = cmr.get_sub_cmap('twilight_shifted', 0.05, 0.6)
        kw_display_pixels1 = dict(pixelsize=dx,
                                  angle=angle_deg,
                                  colorbar=True,
                                  nticks=7,
                                  # cmap='sauron')
                                  cmap=map1)
        # PLOT THE DATA
        #plt.figure()
        #c_c = data[grid[s]] / max(data)
        c_c = data[grid[s]]
        display_pixels.display_pixels(x, y, c_c,
                                      vmin=min(data[grid[s]]), vmax=max(data[grid[s]]),
                                      label='velocity',
                                      **kw_display_pixels1)
        #plt.gca().set_title(title)
        #plt.show()

    fig = plt.figure(figsize=(20, 30))

    n_rows, n_cols = orblib.n_orbs // orb_skip + 1, 3

    ax = plt.subplot(n_rows, n_cols, n_cols * i + 1)
    if len(orblib.losvd_histograms) > 1:  # assuming that 1d histograms are first, if existing
        ax.set_title(f'Orbit {orb_idx}, losvd')
        mean = orblib.losvd_histograms[0].get_mean()[orb_idx]
        create_plot(stars.kinematic_data[0].dp_args, mean)

    idx = 1 if len(orblib.losvd_histograms) > 1 else 0  # assuming that 1d histograms are first, if existing

    ax = plt.subplot(n_rows, n_cols, n_cols * i + 2)
    ax.set_title(f'Orbit {orb_idx}, pm x')
    mean = orblib.losvd_histograms[idx].get_mean()[0][orb_idx]
    create_plot(stars.kinematic_data[idx].dp_args, mean)

    ax = plt.subplot(n_rows, n_cols, n_cols * i + 3)
    ax.set_title(f'Orbit {orb_idx}, pm y')
    mean = orblib.losvd_histograms[idx].get_mean()[1][orb_idx]
    create_plot(stars.kinematic_data[idx].dp_args, mean)

    #ax = plt.subplot(n_rows, n_cols, n_cols, n_cols * i + 4)
    #ax.set_title(f'Orbit {orb_idx}, pm vector field')
